<a href="https://colab.research.google.com/github/renataeugenia/ChallengeAlura_DataScience/blob/main/Challenge_Alura_DataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Alura Data Science - 1º semana 21/11/2022

In [12]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [13]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [14]:
import findspark
findspark.init()

In [15]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Challenge Alura") \
    .getOrCreate()

In [16]:
spark

**descompactando** **arquivos**

In [17]:
import zipfile

In [18]:
zipfile.ZipFile('/content/drive/MyDrive/Challenge_Alura/semana-1.zip', 'r').extractall('/content/drive/MyDrive/Challenge_Alura')

In [19]:
path = '/content/drive/MyDrive/Challenge_Alura/dataset_bruto.json'
dataset_bruto = spark.read.json(path)

**explorando base de dados**

In [20]:
dataset_bruto.count()

89083

In [21]:
dataset_bruto.limit(5).toPandas()

,anuncio,imagens,usuario
0,"(0, [], [16], [0], [], (Centro, 20061003, Rio ...","[(39d6282a-71f3-47bc-94aa-909351ecd881, https:...","(9d44563d-3405-4e84-9381-35b7cf40a9a4, Frank)"
1,"(0, [], [14], [0], [], (Centro, 20051040, Rio ...","[(23d2b3ab-45b0-47f2-a27f-ffbe066c2c38, https:...","(36245be7-70fe-40cd-84d4-70a57dc8f976, Caroline)"
2,"(0, [1026], [1026], [0], [], (Maria da Graça, ...","[(1da65baa-368b-490f-bef7-1834b9a428c1, https:...","(9dc415d8-1397-4d8d-8425-b8310fad309d, Oliver)"
3,"(0, [120], [120], [0], [Portão eletrônico, Con...","[(79b542c6-49b4-4f06-80b5-ef4fcdf98f76, https:...","(9911a2df-f299-4a73-a384-281417abaf69, Matthew)"
4,"(0, [3], [3], [0], [], (São Cristóvão, 2093167...","[(e2bc497b-6510-4731-8942-97b3d3cdab5e, https:...","(240a7aab-12e5-4069-9a2c-9dca4c4f9c68, Jude)"


In [22]:
dataset_bruto.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [26]:
anuncio = dataset_bruto\
.select('anuncio.*')

In [27]:
anuncio.limit(5).toPandas()

,andar,area_total,area_util,banheiros,caracteristicas,endereco,id,quartos,suites,tipo_anuncio,tipo_unidade,tipo_uso,vaga,valores
0,0,[],[16],[0],[],"(Centro, 20061003, Rio de Janeiro, Rio de Jane...",47d553e0-79f2-4a46-9390-5a3c962740c2,[0],[0],Usado,Outros,Comercial,[1],"[(260, 107, Venda, 10000)]"
1,0,[],[14],[0],[],"(Centro, 20051040, Rio de Janeiro, Rio de Jane...",b6ffbae1-17f6-4870-9950-e998ac1c8d6a,[0],[],Usado,Outros,Comercial,[0],"[(260, 107, Venda, 10000)]"
2,0,[1026],[1026],[0],[],"(Maria da Graça, 20785360, Rio de Janeiro, Rio...",1fb030a5-9e3e-4a1d-93cd-cd2d8a215e11,[0],[],Usado,Outros,Comercial,[0],"[(None, 1613, Venda, 10000)]"
3,0,[120],[120],[0],"[Portão eletrônico, Condomínio fechado]","(Campo Grande, 23095071, Rio de Janeiro, Rio d...",ac6cb348-69d6-45af-9589-dc34099370d8,[0],[],Usado,Outros,Residencial,[],"[(80, None, Venda, 10000)]"
4,0,[3],[3],[0],[],"(São Cristóvão, 20931670, Rio de Janeiro, Rio ...",e032b908-ef42-4d4a-8125-eba4792bacbe,[0],[],Usado,Outros,Residencial,[],"[(0, 0, Venda, 5000)]"


In [29]:
anuncio\
.select('tipo_uso')\
.groupby('tipo_uso')\
.count()\
.show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+

